<a href="https://colab.research.google.com/github/android-kunjapppan/Tensorflow-in-Practice/blob/master/NLP_Tensorflow_week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embeddings

* 

In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [3]:
import tensorflow_datasets as tfds

imdb,info = tfds.load("imdb_reviews",with_info=True, as_supervised=True)

In [4]:
import numpy as np

train_data, test_data = imdb['train'],imdb['test']

In [7]:
train_sentences =[]
train_labels = []

test_sentences=[]
test_labels=[]

for s,l in train_data:
  train_sentences.append(s.numpy().decode('utf8'))
  train_labels.append(l.numpy())

for s,l in test_data:
  test_sentences.append(s.numpy().decode('utf8'))
  test_labels.append(l.numpy())

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [8]:
num_words=10000
embedding_dim=16
max_len= 120
trunc_type='post'
oov_tok = "<OOV>"


In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=num_words,oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_sentences)
padded = pad_sequences(sequences,maxlen=max_len,truncating=trunc_type)

test_sequences=tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences,maxlen=max_len)

In [24]:
rev_word_index = dict([(value,key) for (key,value) in word_index.items()])

def decode_review(text):
  return ' '.join([rev_word_index.get(i,'?') for i in text])

print(decode_review(padded[1]))
print(train_sentences[1])

? ? ? ? ? ? ? ? i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the <OOV> and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow and boring things seemed to happen but with no explanation of what was causing them or why i admit i may have missed part of the film but i watched the majority of it and everything just seemed to happen of its own <OOV> without any real concern for anything else i cant recommend this film at all
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what

In [14]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding,Flatten,Dense

model = Sequential([
                    Embedding(num_words, embedding_dim,input_length = max_len),
                    Flatten(),
                    Dense(6, activation ='relu'),
                    Dense(1, activation='sigmoid')
  
])

model.compile(loss='binary_crossentropy', optimizer ='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [15]:
epochs =10
model.fit(padded, train_labels, epochs = epochs,validation_data=(test_padded,test_labels))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.4877 - accuracy: 0.7506 - val_loss: 0.3382 - val_accuracy: 0.8530
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2390 - accuracy: 0.9077 - val_loss: 0.3661 - val_accuracy: 0.8431
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0937 - accuracy: 0.9759 - val_loss: 0.4378 - val_accuracy: 0.8322
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0233 - accuracy: 0.9966 - val_loss: 0.5169 - val_accuracy: 0.8319
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0057 - accuracy: 0.9996 - val_loss: 0.5801 - val_accuracy: 0.8320
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.6319 - val_accuracy: 0.8312
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 8.7521e-04 - accuracy: 1.0000 - val_loss: 0.6715 - val_accuracy: 0.8334
Ep

In [31]:
from tensorflow.keras.layers import GlobalAveragePooling1D
model2 = Sequential([
                    Embedding(num_words, embedding_dim,input_length = max_len),
                    GlobalAveragePooling1D(),
                    Flatten(),
                    Dense(6, activation ='relu'),
                    Dense(1, activation='sigmoid')
  
])

model2.compile(loss='binary_crossentropy', optimizer ='adam', metrics=['accuracy'])
model2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [32]:
epochs =10
model2.fit(padded, train_labels, epochs = epochs,validation_data=(test_padded,test_labels))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.5419 - accuracy: 0.7565 - val_loss: 0.3800 - val_accuracy: 0.8397
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.3252 - accuracy: 0.8654 - val_loss: 0.3307 - val_accuracy: 0.8582
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2651 - accuracy: 0.8934 - val_loss: 0.3329 - val_accuracy: 0.8566
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.2294 - accuracy: 0.9111 - val_loss: 0.3460 - val_accuracy: 0.8520
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2025 - accuracy: 0.9237 - val_loss: 0.3696 - val_accuracy: 0.8467
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.1823 - accuracy: 0.9338 - val_loss: 0.4011 - val_accuracy: 0.8388
Epoch 7/10
782/782 [==============================] - 4s 4ms/step - loss: 0.1640 - accuracy: 0.9426 - val_loss: 0.4285 - val_accuracy: 0.8363
Epoch 

In [18]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [21]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, num_words):
  word = rev_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [22]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]


# Sarcasm

In [39]:
import json

In [40]:
vocab_size = 1000
embedding_dim = 32
max_length = 16
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [41]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json


--2020-07-21 22:10:06--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.214.128, 173.194.216.128, 173.194.217.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.07s   

2020-07-21 22:10:06 (74.3 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [45]:
with open("/tmp/sarcasm.json",'r') as f:
  data= json.load(f)

sentences=[]
labels=[]

for item in data:
  sentences.append(item['headline'])
  labels.append(item['is_sarcastic'])

In [44]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]

training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [47]:
tokenizer = Tokenizer(num_words = vocab_size,oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)


In [48]:
training_padded= np.array(training_padded)
training_labels= np.array(training_labels)
testing_padded= np.array(testing_padded)
testing_padded= np.array(testing_padded)

In [49]:
model3 = Sequential([
                    Embedding(vocab_size, embedding_dim,input_length = max_length),
                    GlobalAveragePooling1D(),
                    Dense(24, activation ='relu'),
                    Dense(1, activation='sigmoid')
  
])

model3.compile(loss='binary_crossentropy', optimizer ='adam', metrics=['accuracy'])
model3.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 16, 32)            32000     
_________________________________________________________________
global_average_pooling1d_3 ( (None, 32)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 24)                792       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 25        
Total params: 32,817
Trainable params: 32,817
Non-trainable params: 0
_________________________________________________________________


In [50]:
num_epochs = 30
history = model3.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/30


UnboundLocalError: ignored